##### Importing all necessary library to prepare dataset directory

In [2]:
import os
import shutil
import random

### Creating Dataset with 80% of data for training and 20% for validation 

In [3]:
# creating folders of validation set
os.chdir("./dataset_2/val")
for i in range(ord("A"),ord("Z")+1):
    os.mkdir(chr(i))
os.chdir("../")


In [4]:
# transfering 20% of image to validation set
for i in range(ord("A"),ord("Z")+1):
    # select random 5 image out of 50 image
    validation_set = random.sample(os.listdir(f"train/{chr(i)}"),5)
    for j in validation_set:
        shutil.move(f"train/{chr(i)}/{j}",f"val/{chr(i)}")

In [5]:
# come back to main directory
os.chdir("../")

In [6]:
# defining train and validation dataset directory
train_path = "dataset_2/train"
valid_path = "dataset_2/val"

##### Importing all necessary library and module for ASL ditection model training 

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input,MobileNet,decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

#### Preparing training and validation batches

In [8]:
train_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    directory = train_path, target_size=(300,300), batch_size=10,class_mode="categorical"
)
valid_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    directory = valid_path, target_size=(300,300), batch_size=10,class_mode="categorical"
)

Found 1263 images belonging to 26 classes.
Found 130 images belonging to 26 classes.


#### Building a Transfer learning model from pretrained mobilenet model 

In [33]:
# get mobile_net_v3_small model with input shape (300,300,3)
mobilenet_model = tf.keras.applications.MobileNetV3Small(input_shape=(300,300,3))

In [1]:
# view all layer of mobilenet model and its parameters
mobilenet_model.summary()

NameError: name 'mobilenet_model' is not defined

In [40]:
# add a sofmax Dense layer to mobilenet model
x = mobilenet_model.layers[-2].output
output = Dense(26,activation="softmax")(x)

In [3]:
# Build a transfer learing model from mobilenet model
model = Model(inputs=mobilenet_model.input, outputs=output)

NameError: name 'Model' is not defined

In [42]:
# make last 23 layer only trainable
for layer in model.layers[:-20]:
    layer.trainable = False

In [37]:
# view new transfer learning models layers
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 300, 300, 3)  0           ['input_3[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 150, 150, 16  432         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                            

#### Train the model

In [43]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

In [45]:
# train the model
model.fit(x=train_batches ,validation_data=valid_batches,epochs=15)

Epoch 1/15
127/127 [==============================] - 55s 434ms/step - loss: 0.0889 - accuracy: 0.9747 - val_loss: 1.0769 - val_accuracy: 0.6308
Epoch 2/15
127/127 [==============================] - 57s 445ms/step - loss: 0.0706 - accuracy: 0.9810 - val_loss: 0.9220 - val_accuracy: 0.7077
Epoch 3/15
127/127 [==============================] - 56s 443ms/step - loss: 0.0795 - accuracy: 0.9755 - val_loss: 0.6277 - val_accuracy: 0.7769
Epoch 4/15
127/127 [==============================] - 57s 444ms/step - loss: 0.0569 - accuracy: 0.9865 - val_loss: 0.5312 - val_accuracy: 0.8077
Epoch 5/15
127/127 [==============================] - 57s 448ms/step - loss: 0.0421 - accuracy: 0.9905 - val_loss: 0.4793 - val_accuracy: 0.8231
Epoch 6/15
127/127 [==============================] - 58s 455ms/step - loss: 0.0416 - accuracy: 0.9913 - val_loss: 0.4366 - val_accuracy: 0.7923
Epoch 7/15
127/127 [==============================] - 55s 430ms/step - loss: 0.0353 - accuracy: 0.9929 - val_loss: 0.3965 - val_ac

In [10]:
# save model
model.save("model_2.h5")

In [5]:
model = load_model("model.h5")

In [16]:
img = image.load_img("./Dataset/validation/V/1.png",target_size=(300,300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
predictions = model.predict(x)

1/1 [==============================] - 0s 80ms/step


In [17]:
np.argmax(predictions[0],axis=0)

21